# p-平均法（撰寫中）

本文介紹作者提出的 p-平均法。

針對交易表現的檢定，常見作法是使用 t 檢定；
或採用《ファイナンス機械学習》提出的 PSR（Probabilistic Sharpe Ratio）、DSR（Deflated Sharpe Ratio）等。

這些方法的問題在於對「報酬的長期變化」較不敏感。
例如三年前顯著獲利，但近一年為負，未來是否能獲利就很可疑；
然而若使用上述方法，可能仍被判定為「穩定可獲利」，
原因在於它們未考慮樣本的時間順序（順序資訊未被利用）。

為緩解此問題，提出 p-平均法，做法如下（p 值平均越低越好）：

1. 將報酬時間序列切成 N 段
2. 在各段內進行 t 檢定以取得 p 值
3. 取 N 個 p 值的平均
4. 以 p 值平均作為判定準則

直覺上，若存在任何一段 p 值很大，平均後的 p 值也會偏大，
因此只有在「所有期間」皆表現穩定時才會被判定為顯著，
有助於避免只依賴舊期間的高報酬而忽視近期惡化。

實作上，`calc_p_mean` 負責計算 p-平均法；
型一錯誤率（Type I error rate）可用 `calc_p_mean_type1_error_rate` 計算。

### 為什麼 p-平均法能緩解問題

當報酬具長期變化時，直接套用 t 檢定並不合適，
因為 t 檢定假設樣本為「同一分布且獨立同分布」。
雖然 p-平均法最終仍使用 t 檢定，但由於是在較短的子期間中進行，
更容易滿足「同一分布」的假設，能部分緩解不適配的問題。

### 關於分布移轉（Distribution Shift）

p-平均法以期間分段檢定，可跨不同市場階段（牛市、熊市、極端事件等）進行。

例如：若訓練資料未包含類似「雷曼兄弟」的事件，實務上遇到該情境便可能虧損。
可設計壓力情境（Stress Scenario）並以子期間評估，以降低此風險。

### 改進要點

#### 含浮動虧損的強制歸零（Zero-cut）

為求簡潔，本文未將「含浮虧的歸零處理」納入；
可改寫回測程式支援此機制，亦可用於推估適當的槓桿大小。
